In [1]:
#### import ssdeep
import pickle,os,sys,gc
import pandas as pd
# pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm
from collections import Counter
import multiprocessing as mp
from multiprocessing import Process, Manager, Pool
import subprocess
from multiprocessing import Pool 
import time,datetime
from itertools import chain
import math
from operator import itemgetter
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import functools
import itertools
import operator
from multiprocessing.dummy import Pool as ThreadPool
import os.path
from os import path
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
protocols_need = ['ssh','smb','http','telnet','smtp','ftp','sip','mysql','rpc','pop','imap']
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
protos_pktinfo_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict_sampled_pktinfo_123.pkl','rb'))
protos_pktinfo_df['http']

In [ ]:
def count_statistics(df,seriesname):
    '''
    GOAL: count 1D list's statistics
    '''
    list1d = df[seriesname]
    return (min(list1d),np.quantile(list1d,0.25),np.quantile(list1d,0.5),np.quantile(list1d,0.75),
            max(list1d),np.mean(list1d),np.std(list1d))
def count_statistics_payloadinfo(df,seriesname):
    '''
    GOAL: count statistics for nested(3D) structure list
    '''
    list3d = df[seriesname]
    min_li=[]
    q1_li=[]
    q2_li=[]
    q3_li=[]
    max_li=[]
    avg_li=[]
    std_li = []
    for list2d in list3d:
        records = pd.DataFrame.from_records(list2d)
        records[2] = records[2].astype(float)
        min_li.append(records[2].min())
        q1_li.append(records[2].quantile(0.25))
        q2_li.append(records[2].quantile(0.5))        
        q3_li.append(records[2].quantile(0.75))
        max_li.append(records[2].max())
        avg_li.append(records[2].mean())
        std_li.append(records[2].std())
#     return (min_li,q1_li,q2_li,q3_li,max_li,
#             avg_li,std_li)        
    return (np.nanmean(min_li),np.nanmean(q1_li),np.nanmean(q2_li),np.nanmean(q3_li),np.nanmean(max_li),
            np.nanmean(avg_li),np.nanmean(std_li))

In [ ]:

# gc.collect()        
for proto in tqdm(protocols_need):
    proto_df = protos_pktinfo_df[proto]
    proto_df = proto_df[proto_df['pkt_allinfo'].apply(len)>2]
    proto_df = proto_df[['idx','total_packets_num','total_sessions_num','total_ip_num',
                     'total_country_num','pkt_allinfo']]
    proto_df = proto_df.sort_values('total_sessions_num',ascending=False)
    proto_df = proto_df.reset_index(drop=True)
    proto_df[['sampled_timelist','sampled_sess_duration','sampled_i_tt_pkts','sampled_i_tt_payload_length',
             'sampled_i_avg_payload_length','sampled_i_payloadinfo']] = pd.DataFrame(proto_df.pkt_allinfo.tolist(),index=proto_df.index)
    sampled_df = proto_df#[proto_df['pkt_allinfo'].apply(len)>2]
    
    all_names = ['idx','total_sessions_num','total_packets_num','total_ip_num','total_country_num']
    stat_li = ['min','Q1','Q2','Q3','max','avg','std']
    name_li = ['sess_duration_'+x for x in stat_li]
    all_names.extend(name_li)
    sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_sess_duration',),axis=1,result_type='expand')

    name_li = ['sess_pktsnum_'+x for x in stat_li]
    all_names.extend(name_li)
    sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_tt_pkts',),axis=1,result_type='expand')

    name_li = ['sess_inpayloadtt_'+x for x in stat_li]
    all_names.extend(name_li)
    sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_tt_payload_length',),axis=1,result_type='expand')

    name_li = ['avgpkt_inpayload_'+x for x in stat_li]
    all_names.extend(name_li)
    sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_avg_payload_length',),axis=1,result_type='expand')

    name_li = ['pkt_inpayload_'+x+'avg' for x in stat_li]
    all_names.extend(name_li)
    sampled_df[name_li] = sampled_df.apply(count_statistics_payloadinfo,args=('sampled_i_payloadinfo',),axis=1,result_type='expand')
    # sampled_df['pkt_inpayload_minavg'] = sampled_df['sampled_i_payloadinfo'].apply() #先找session內封包的Q1Q2以後，再和其他session的Q1Q2...取平均

    # tmp['pkt_allinfo'][56] 
    output_excel = sampled_df[all_names]
    output_excel.to_excel('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pictures/crossisp_statistics_dfdict_sampled_pktinfo_'+proto+'.xlsx')    
sampled_df

In [4]:
# proto_df = protos_pktinfo_df['http']
# proto_df[proto_df['pkt_allinfo'].apply(len)>2]

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,...,date_count,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,taken,flat_timestamp,pkt_allinfo
56,20200106_4208_http,"[1578241737.276672, 1578243643.079254, 1578252...","[Canada, Canada, Canada, Canada, Canada, Taiwa...","[172.105.4.227, 172.105.4.227, 172.105.4.227, ...",[3:pvEd+aCKa5W/A+X+UfWrfWBTsXf2Lun:1g/Cr5W/A+u...,"[(43.6547, -79.3623), (43.6547, -79.3623), (43...",台灣之星,http,69,[3:pvEd+aCKa5W/A+X+UfWrfWBTsXf2Lun:1g/Cr5W/A+u...,...,{'20200106': 69},0.0,{'台灣之星_20200106': 69},44166,69,2,2,True,"[1578241737.276672, 1578243643.079254, 1578252...","[{'台灣之星_20200106': [[1578285351.406782, 157828..."
182,20200110_42949677222_http,"[1578651482.181124, 1578651482.571463, 1578651...","[Germany, Germany, Germany, Germany, Germany, ...","[87.133.23.192, 87.133.23.192, 87.133.23.192, ...",[24:tAUsTLLawnUHDOVJrckfALJt1IpRPafCdF:+XfLaDm...,"[(52.541, 9.7429), (52.541, 9.7429), (52.541, ...",台灣大哥大,http,7,[24:tAUsTLLawnUHDOVJrckfALJt1IpRPafCdF:+XfLaDm...,...,{'20200110': 7},0.0,{'台灣大哥大_20200110': 7},1793,7,1,1,False,"[1578651482.181124, 1578651482.571463, 1578651...","[{'台灣大哥大_20200110': [[1578651491.570963, 15786..."
252,20200106_13656_http_20200106_2984_http_2020010...,"[[1578239437.622562, 1578239439.00718, 1578239...","[[United States, United States, United States,...","[[66.249.65.188, 66.249.79.137, 66.249.71.43, ...",[[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691u...,"[[(37.751, -97.822), (37.751, -97.822), (37.75...","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電...","[http, http, http, http, http, http, http, htt...","[183041, 64609, 18631, 1062, 62, 2, 326422, 86...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,...,"{'20200105': 1832, '20200106': 364279, '202001...",0.0,"{'亞太電信_20200106': 57038, '亞太電信_20200107': 3356...",2900868695,2735459,44486,178,True,"[1578239437.622562, 1578239439.00718, 15782394...","[{'台灣大哥大_20200110': [[1578635655.619146, 15786..."
253,20200106_295_http_20200107_35988_http_20200106...,"[[1578239440.513196, 1578239443.592105, 157823...","[[Iran, Russia, United States, United States, ...","[[79.132.201.185, 212.41.14.170, 216.244.66.24...",[[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da...,"[[(35.698, 51.4115), (56.003, 92.7993), (47.72...","[亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電信, 中華電信, 中華電信, 中華電...","[http, http, http, http, http, http, http, htt...","[81808, 2169, 80, 268588, 130847, 9758, 15, 7,...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,...,"{'20200105': 1283, '20200106': 150793, '202001...",0.0,"{'亞太電信_20200106': 9327, '亞太電信_20200107': 9635,...",1041512399,1188705,94601,189,True,"[1578239440.513196, 1578239443.592105, 1578239...","[{'台灣大哥大_20200110': [[1578671346.860649, 15786..."
254,20200106_51539658953_http_20200106_13915_http_...,"[[1578243129.379452, 1578243147.033231, 157824...","[[China, China, China, China, China, China, Ho...","[[103.140.126.152, 103.140.126.152, 103.140.12...",[[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1Jc...,"[[(34.7725, 113.7266), (34.7725, 113.7266), (3...","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電信, 中華電信, 台灣基...","[http, http, http, http, http, http, http, htt...","[48240, 882, 66, 11, 1788, 1345, 158, 67071, 1...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,...,"{'20200106': 99634, '20200107': 87619, '202001...",0.0,"{'亞太電信_20200106': 384, '亞太電信_20200107': 3196, ...",542600338,541788,36956,177,True,"[1578243129.379452, 1578243147.033231, 1578243...","[{'台灣之星_20200106': [[1578270970.238054, 157827..."
255,20200106_152_http_20200106_8589947030_http_202...,"[[1578239434.511962, 1578239437.425728, 157823...","[[China, China, China, China, China, China, Ch...","[[122.14.208.63, 139.199.74.166, 148.70.163.11...",[[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:z...,"[[(23.1167, 113.25), (34.7725, 113.7266), (34....","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電...","[http, http, http, http, http, http, http, htt...","[31897, 583, 306, 251, 226, 15

In [5]:
# proto_df.columns

Index(['idx', 'timestamp', 'country', 'src_ip', 'ssdeep', 'lalo', 'isp',
       'protocol', 'session_num', 'ssdeep_flat', 'signature', 'ssdeep_clean',
       'clusterkey_timelist', 'date_count', 'sum', 'isp_date_sessioncount',
       'total_packets_num', 'total_sessions_num', 'total_ip_num',
       'total_country_num', 'taken', 'flat_timestamp', 'pkt_allinfo'],
      dtype='object')

In [6]:
# proto_df = proto_df[['idx','total_packets_num','total_sessions_num','total_ip_num',
#                      'total_country_num','pkt_allinfo']]
# proto_df ##dict: 2D list : sessions' time、sampled_sess_duration 1D list、sampled_i_tt_pkts 1D list、
#sampled_i_tt_payload_length 1D list、sampled_i_avg_payload_length 1D list、sampled_i_payloadinfo 3D list: ssdeep,time,length

,idx,total_packets_num,total_sessions_num,total_ip_num,total_country_num,pkt_allinfo
0,20200108_8589990159_http,21036,118,3,3,"[, ]"
1,20200106_8589962872_http,5434,25,1,1,"[, ]"
2,20200106_206158498684_http,441,2,1,1,"[, ]"
3,20200109_128849077624_http,346,2,2,1,"[, ]"
4,20200106_534_http,31210,36,12,7,"[, ]"
...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,15332,11,2,2,"[, ]"
340,20200107_206158449329_http_20200106_1202590851...,14527,11,10,7,"[, ]"
341,20200109_85899367448_http_20200109_8589941749_...,7247,5,1,1,"[, ]"
342,20200110_128849030827_http_20200107_3092376673...,7356,7,2,1,"[, ]"


In [11]:
# proto_df[['sampled_timelist','sampled_sess_duration','sampled_i_tt_pkts','sampled_i_tt_payload_length',
#          'sampled_i_avg_payload_length','sampled_i_payloadinfo']] = pd.DataFrame(proto_df.pkt_allinfo.tolist(),index=proto_df.index)
# proto_df

/home/antslab/.local/lib/python3.7/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,idx,total_packets_num,total_sessions_num,total_ip_num,total_country_num,pkt_allinfo,sampled_timelist,sampled_sess_duration,sampled_i_tt_pkts,sampled_i_tt_payload_length,sampled_i_avg_payload_length,sampled_i_payloadinfo
0,20200108_8589990159_http,21036,118,3,3,"[, ]",,,None,None,None,None
1,20200106_8589962872_http,5434,25,1,1,"[, ]",,,None,None,None,None
2,20200106_206158498684_http,441,2,1,1,"[, ]",,,None,None,None,None
3,20200109_128849077624_http,346,2,2,1,"[, ]",,,None,None,None,None
4,20200106_534_http,31210,36,12,7,"[, ]",,,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,15332,11,2,2,"[, ]",,,None,None,None,None
340,20200107_206158449329_http_20200106_1202590851...,14527,11,10,7,"[, ]",,,None,None,None,None
341,20200109_85899367448_http_20200109_8589941749_...,7247,5,1,1,"[, ]",,,None,None,None,None
342,20200110_128849030827_http_20200107_3092376673...,7356,7,2,1,"[, ]",,,None,None,None,None


In [60]:
# def count_statistics(df,seriesname):
#     '''
#     GOAL: count 1D list's statistics
#     '''
#     list1d = df[seriesname]
#     return (min(list1d),np.quantile(list1d,0.25),np.quantile(list1d,0.5),np.quantile(list1d,0.75),
#             max(list1d),np.mean(list1d),np.std(list1d))
# def count_statistics_payloadinfo(df,seriesname):
#     '''
#     GOAL: count statistics for nested(3D) structure list
#     '''
#     list3d = df[seriesname]
#     min_li=[]
#     q1_li=[]
#     q2_li=[]
#     q3_li=[]
#     max_li=[]
#     avg_li=[]
#     std_li = []
#     for list2d in list3d:
#         records = pd.DataFrame.from_records(list2d)
#         records[2] = records[2].astype(float)
#         min_li.append(records[2].min())
#         q1_li.append(records[2].quantile(0.25))
#         q2_li.append(records[2].quantile(0.5))        
#         q3_li.append(records[2].quantile(0.75))
#         max_li.append(records[2].max())
#         avg_li.append(records[2].mean())
#         std_li.append(records[2].std())
# #     return (min_li,q1_li,q2_li,q3_li,max_li,
# #             avg_li,std_li)        
#     return (np.nanmean(min_li),np.nanmean(q1_li),np.nanmean(q2_li),np.nanmean(q3_li),np.nanmean(max_li),
#             np.nanmean(avg_li),np.nanmean(std_li))
        

In [61]:
# sampled_df = proto_df[proto_df['pkt_allinfo'].apply(len)>2]
# all_names = ['idx']
# stat_li = ['min','Q1','Q2','Q3','max','avg','std']
# name_li = ['sess_duration_'+x for x in stat_li]
# all_names.extend(name_li)
# sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_sess_duration',),axis=1,result_type='expand')

# name_li = ['sess_pktsnum_'+x for x in stat_li]
# all_names.extend(name_li)
# sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_tt_pkts',),axis=1,result_type='expand')

# name_li = ['sess_inpayloadtt_'+x for x in stat_li]
# all_names.extend(name_li)
# sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_tt_payload_length',),axis=1,result_type='expand')

# name_li = ['avgpkt_inpayload_'+x for x in stat_li]
# all_names.extend(name_li)
# sampled_df[name_li] = sampled_df.apply(count_statistics,args=('sampled_i_avg_payload_length',),axis=1,result_type='expand')

# name_li = ['pkt_inpayload_'+x+'avg' for x in stat_li]
# all_names.extend(name_li)
# sampled_df[name_li] = sampled_df.apply(count_statistics_payloadinfo,args=('sampled_i_payloadinfo',),axis=1,result_type='expand')
# # sampled_df['pkt_inpayload_minavg'] = sampled_df['sampled_i_payloadinfo'].apply() #先找session內封包的Q1Q2以後，再和其他session的Q1Q2...取平均

# # tmp['pkt_allinfo'][56] 
# output_excel = sampled_df[all_names]
# output_excel.to_excel(''+proto+'.xlsx')

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice
/home/antslab/.local/lib/python3.7/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,idx,total_packets_num,total_sessions_num,total_ip_num,total_country_num,pkt_allinfo,sampled_timelist,sampled_sess_duration,sampled_i_tt_pkts,sampled_i_tt_payload_length,sampled_i_avg_payload_length,sampled_i_payloadinfo,pkt_inpayload_minavg,pkt_inpayload_Q1avg,pkt_inpayload_Q2avg,pkt_inpayload_Q3avg,pkt_inpayload_maxavg,pkt_inpayload_avgavg,pkt_inpayload_stdavg
56,20200106_4208_http,44166,69,2,2,"[{'台灣之星_20200106': [[1578285351.406782, 157828...","{'台灣之星_20200106': [[1578285351.406782, 1578285...","[0.5341191291809082, 0.9094440937042236, 11.08...","[6, 6, 230, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6...","[50, 50, 153, 50, 50, 50, 50, 50, 50, 50, 50, ...","[8.333333333333334, 8.333333333333334, 0.66521...","[[[3:c57cfQffTGX105tD+U/nE:a7cfQE10HKAE, 15782...",34.275362,34.275362,34.275362,34.275362,34.275362,34.275362,0.000000
182,20200110_42949677222_http,1793,7,1,1,"[{'台灣大哥大_20200110': [[1578651491.570963, 15786...","{'台灣大哥大_20200110': [[1578651491.570963, 157865...","[7.506525039672852, 4.997946739196777, 5.26322...","[5, 5, 5, 5, 5, 5, 5]","[287, 334, 286, 311, 286, 270, 287]","[57.4, 66.8, 57.2, 62.2, 57.2, 54.0, 57.4]",[[[24:tAUsTLLawnUHDOVJrckfALJt1IpRPafCdF:+XfLa...,294.428571,294.428571,294.428571,294.428571,294.428571,294.428571,NaN
252,20200106_13656_http_20200106_2984_http_2020010...,2900868695,2735459,44486,178,"[{'台灣大哥大_20200110': [[1578635655.619146, 15786...","{'台灣大哥大_20200110': [[1578635655.619146, 157863...","[19.935501098632812, 2.0749261379241943, 37.52...","[10, 5, 6, 447, 5, 6, 6, 6, 6, 6, 3837, 6, 6, ...","[2904, 218, 223, 200, 384, 510, 516, 517, 506,...","[290.4, 43.6, 37.166666666666664, 0.4474272930...",[[[24:f5amk6H9AYOsuS18YH+w18YHYKafCq7tRHnvHHhl...,487.164571,488.455714,488.650629,488.680426,489.249317,488.476576,45.841048
253,20200106_295_http_20200107_35988_http_20200106...,1041512399,1188705,94601,189,"[{'台灣大哥大_20200110': [[1578671346.860649, 15786...","{'台灣大哥大_20200110': [[1578671346.860649, 157867...","[1.5003039836883545, 22.996612548828125, 27.33...","[6, 6, 15, 6, 10, 6, 5, 5, 6, 5, 5, 6, 7, 5, 6...","[392, 191, 1337, 191, 768, 192, 322, 192, 192,...","[65.33333333333333, 31.833333333333332, 89.133...",[[[24:1/naZafCe/XESNoxt+tAzx7wSlV0/wDNVKC/:tna...,226.898990,226.898990,226.898990,226.898990,226.898990,226.898990,0.000000
254,20200106_51539658953_http_20200106_13915_http_...,542600338,541788,36956,177,"[{'台灣之星_20200106': [[1578270970.238054, 157827...","{'台灣之星_20200106': [[1578270970.238054, 1578270...","[1.138956069946289, 4.423031806945801, 3.34896...","[5, 8254, 5, 5, 197, 6, 6, 7, 106, 5, 5, 5, 6,...","[275, 268, 228, 225, 228, 240, 233, 225, 540, ...","[55.0, 0.032469105888054274, 45.6, 45.0, 1.157...",[[[24:7jzzuCU4FReQovt1GNYKP+9Cy9Dj4LSCYx:7jXuC...,262.928571,262.928571,262.928571,262.928571,262.928571,262.928571,0.000000
255,20200106_152_http_20200106_8589947030_http_202...,411489900,264115,8693,128,"[{'台灣之星_20200106': [[1578249500.252637, 157824...","{'台灣之星_20200106': [[1578249500.252637, 1578249...","[30.940441131591797, 2.4524519443511963, 1.982...","[10, 6, 7, 6, 196, 9, 6, 6, 253, 277, 63, 7, 6...","[206, 243, 191, 102, 229, 103, 228, 87, 247, 2...","[20.6, 40.5, 27.285714285714285, 17.0, 1.16836...",[[[6:rHKA2y6A7GyGTSjkRB9LyhvXfLTYCSUBKAQNWjyGT...,202.166357,202.602659,203.038961,203.475263,203.911565,203.038961,7.309360
257,20200106_1407_http_20200108_2136_http_20200106...,122956233,150594,17256,147,"[{'台灣大哥大_20200110': [[1578621270.043933, 15786...","{'台灣大哥大_20200110': [[1578621270.043933, 157862...","[18.07757568359375, 13.950279235839844, 19.406...","[13, 9, 14, 8, 7, 6, 6, 7, 16, 5, 6, 6, 11, 11...","[108, 22, 126, 54, 39, 18, 18, 36, 144, 38, 18...","[8.307692307692308, 2.4444444444444446, 9.0, 6...","[[[3:cRm5tD+U/gfun:rHKAgmn, 1578621271.1346500...",23.390777,23.941141,24.430825,24.841626,25.247573,24.367638,3.111958
258,20200106_16231_http_20200106_369367187960_http...,3322146,14496,1713,79,"[{'台灣大哥大_20200110': [[1578585161.977523, 15785...","{'台灣大哥大_20200110': [[157

In [59]:
# np.nanmean([np.nan,np.nan,0])

# proto_df = proto_df.sort_values('total_sessions_num',ascending=False)
# proto_df = proto_df.reset_index(drop=True)

# temp = tmp['sampled_i_payloadinfo'][252][2]
# kk = pd.DataFrame.from_records(temp)
# kk[2] = kk[2].astype(float)
# kk[2].quantile(.5)

0.0

In [11]:
# class Solution:
def generate(numRows: int) :
    final_li = []
    if numRows==1:
        return [[1]]
    elif numRows==2:
        return [[1],[1,1]]
    else:
        final_li = [[1],[1,1],[1,2,1]]
        for i in range(numRows-3):
            last_li = final_li[-1][:]
            last_li.insert(0,0)
            last_li.append(0)
            now_result = []
            for i,di in enumerate(last_li):
                sum_result = di+last_li[i+1]
                now_result.append(sum_result)
                if i==len(last_li)-2:
                    break     
#             now_result.append(1)
            final_li.append(now_result)
        return final_li
                
#         return final_li
#     for i in range(numRows):
#         if i==0:
#             final_li.append([1])
#         else:
#             for li in final_li:
#                 li.insert(0,0)
#                 li.append(0)
#                 now_result=[]
#                 for i,di in enumerate(li):
#                     sum_result = di+li[i+1]
#                     now_result.append(sum_result)
#                     if i==len(li)-2:
#                         break
#                 now_result.append(1)
#                 final_li.append(now_result)
#     return final_li
generate(numRows=6)

[[1], [1, 1], [1, 2, 1], [1, 3, 3, 1], [1, 4, 6, 4, 1], [1, 5, 10, 10, 5, 1]]

In [ ]:
for i in range(99999):
    a+b+b=d
    a=i
    st_a = str(i)
    if len(st_a)!=5:
        continue
    else:
        for j in range(100):
            if len(str(j))!=2:
                continue
            vw=j
            t = (st_a[-2])
            b = int(str(t)+str(vw))
            d = a+b+b
            st_z = str(d)
            if st_z[-2]!=st_a[-2]:
                continue
            if st_z[-1]==st_a[-1]:
                continue
            
    a

In [ ]:
def partition(n):
    for i in range()
    
    return value
value = partition(5)
print(value)  #此行不要刪
